In [14]:
%cd /data/codes/apa/kaldi/g2p

import pandas as pd
import os
from glob import glob
import json
from pandarallel import pandarallel
import random
import re

pandarallel.initialize(nb_workers=10, progress_bar=True)

/data/codes/apa/kaldi/g2p
INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [15]:
json_dir = "/data/audio_data/pronunciation_scoring_result/marking_data/10"
audio_dir = "/data/audio_data/prep_submission_audio/10"
metadata_path="/data/audio_data/pronunciation_scoring_result/dataset/info_qt_10_trainset.csv"

# json_dir = "/data/audio_data/pronunciation_scoring_result/marking_data/12"
# audio_dir = "/data/audio_data/prep_submission_audio/12"
# metadata_path="/data/codes/pronunciation-scoring-evaluate/data/raw/train-data-type-12-v2.csv"

metadata = pd.read_csv(metadata_path)
print(metadata.shape)
metadata = metadata[metadata.total_time > 1.0]
print(metadata.shape)
metadata.head(2)

(1509318, 12)
(1481822, 12)


,id,is_deleted,user_id,question_id,question_type,question_content,url,score,fidelity_class,created_at,total_time,word_count
0,5580001,0,88226.0,26144,10,Seat,https://storage.googleapis.com/materials-eleme...,53.00,RELEVANT,2023-09-18 21:17:11,2.45,1.0
1,5580002,0,63452.0,107048,10,Keep your body in shape,https://storage.googleapis.com/materials-eleme...,67.46,RELEVANT,2023-09-18 21:17:12,3.05,5.0


In [16]:
def parse_metadata_data(json_path):
    try: 
        with open(json_path, "r") as f:
            content = json.load(f)
        id = os.path.basename(json_path).split(".")[0]

        utterances = []
        for raw_utterance in content["utterance"]:
            
            for word_id, word in enumerate(raw_utterance["words"]):
                
                trans_arpabet = word["trans_arpabet"]
                text = word["text"]

                utterances.append([text, trans_arpabet])
        
        return utterances
    except:
        return []

extracted_data = metadata.id.parallel_apply(lambda x: parse_metadata_data(os.path.join(json_dir, f'{x}.json')))
extracted_data.head()
# metadata.id.head(100).apply(lambda x: parse_metadata_data(os.path.join(json_dir, f'{x}.json')))

0                                    [[SEAT, S IY1 T]]
1    [[KEEP, K IY1 P], [YOUR, Y UH1 R], [BODY, B AA...
2                                [[SCHOOL, S K UW1 L]]
3    [[THIS, DH IH1 S], [IS, IH1 Z], [A, AH0], [GOO...
5                [[PRECISION, P R IH0 S IH1 ZH AH0 N]]
Name: id, dtype: object

In [17]:
lexicon = extracted_data.explode().reset_index()
lexicon.dropna(inplace=True)


In [18]:
lexicon = extracted_data.explode().reset_index()
lexicon.dropna(inplace=True)

lexicon["word"] = lexicon["id"].apply(lambda x: x[0])
lexicon["arpa"] = lexicon["id"].apply(lambda x: x[1])

In [19]:
from tqdm import tqdm

count = {}
vocab = {}
for index in tqdm(lexicon.index):
    word = lexicon["word"][index].upper()
    arpa = lexicon["arpa"][index]
    key = f'{word}-{arpa}'

    if key not in count:
        count[key] = 1
    else:
        count[key] += 1

    if word not in vocab:
        vocab[word] = [arpa, ]
    else:
        if arpa not in vocab[word]:
            vocab[word].append(arpa)
    

  0%|          | 0/5399442 [00:00<?, ?it/s]

100%|██████████| 5399442/5399442 [00:30<00:00, 175659.47it/s]


In [20]:
lexicon = []
for key, value in vocab.items():
    if len(key.split()) > 1:
        continue
    
    for arpa in value:
        tmp = f'{key}-{arpa}'
        # if count[tmp] < 2:
        #     continue
        lexicon.append([key, arpa])
        
len(lexicon)

3667

In [21]:
path = "/data/codes/apa/kaldi/g2p/lexicon/processed/elsa-lexicon-10.txt"
with open(path, "w", encoding="utf-8") as f:
    for line in lexicon:
        f.write("\t".join(line))
        f.write("\n")